In [ ]:
!pip install flask
!pip install flask_ngrok
!pip install apex
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4
!apt-get install xpdf

In [ ]:
mkdir -p data/amazon

In [ ]:
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q4/Amazon-Q4-2020-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/Q1/AMZN-Q1-2020-Earnings-Release.pdf

In [ ]:
ls data/amazon

In [ ]:
!nvidia-smi

In [ ]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 40

In [ ]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(scheme="https",host=HOST,port=9243, username=USER, password=PWD, index="document")

In [ ]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [ ]:

directory = "data"
import os
from haystack.file_converter.pdf import PDFToTextConverter
from haystack.preprocessor.preprocessor import PreProcessor

converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["de","en"])

processor = PreProcessor(clean_empty_lines=True,
                         clean_whitespace=True,
                         clean_header_footer=True,
                         split_by="word",
                         split_length=200,
                         split_respect_sentence_boundary=True)
docs = []
for filename in os.listdir(directory):

    # Run the conversion on each file (PDF -> 1x doc)
    d = converter.convert(os.path.join(directory, filename), meta={"name":filename})

    # clean and split each dict (1x doc -> multiple docs)
    d = processor.process(d)
    docs.extend(d)

# Let's have a look at the first 3 entries:
print(docs[:3])

In [ ]:
len(docs)

In [ ]:
document_store.write_documents(docs)

In [ ]:
 from haystack.retriever.dense import DensePassageRetriever
 retriever = DensePassageRetriever(document_store=document_store,
                                      query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                      passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                      max_seq_len_query=64,
                                      max_seq_len_passage=256,
                                      batch_size=2,
                                      use_gpu=True,
                                      embed_title=True,
                                      use_fast_tokenizers=True
                                      )

In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:
document_store.update_embeddings(retriever)

In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [ ]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(query="Who is a counterparty?", top_k_retriever=5, top_k_reader=5)

In [ ]:
print_answers(prediction, details="minimal")

In [ ]:
import json
import os
import logging
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from flask import Flask, request, jsonify
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts
from haystack.reader.farm import FARMReader
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.file_converter.pdf import PDFToTextConverter
from haystack.retriever.dense import DensePassageRetriever
from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.pipeline import ExtractiveQAPipeline
from werkzeug.utils import secure_filename

#application settings
app = Flask(__name__)
CORS(app)


In [ ]:
@app.route('/query',methods=['GET', 'POST'])
def search():
    """Return the n answers."""

    question = request.get_json()
    question = question['questions']

    prediction = pipe.run(query=question[0], top_k_retriever=3, top_k_reader=3)
    answer = []
  
    for res in prediction['answers']:
        answer.append(res['answer'])

    result = {"results":[prediction]}
    return json.dumps(result)


In [ ]:
@app.route('/file-upload', methods = ['GET', 'POST'])
def upload_file():
   if request.method == 'POST':
      f = request.files['file']
      f.save(secure_filename(f.filename))
      return 'File Uploaded Successfully'

In [ ]:
run_with_ngrok(app)
app.run()

In [ ]:
from haystack.connector import Crawler
    
crawler = Crawler()
# crawl Haystack docs, i.e. all pages that include haystack.deepset.ai/docs/
docs = crawler.crawl(urls=["https://haystack.deepset.ai/docs/latest/get_startedmd"],
    output_dir="crawled_files",
    filter_urls= ["haystack\.deepset\.ai\/docs\/"])